In [1]:
from matplotlib.axes import Axes
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys

from search_functions_v2 import flooding, random_walk, instant_random_walk
from validate import read_and_validate_network


In [2]:
G = read_and_validate_network("grafo.txt")
cache_dfs = {}
cache_bfs = {}


Graph with 10 nodes and 12 edges
O grafo é válido!


In [3]:
%matplotlib qt
pos: dict = nx.spring_layout(G)

for node in G.nodes():
    if 'color' not in G.nodes[node]:
        G.nodes[node]['color'] = 'lightblue'

for edge in G.edges():
    if 'color' not in G.edges[edge]:
        G.edges[edge]['color'] = 'black'

ax: Axes
_, ax = plt.subplots(figsize=(8, 6))

def draw(G: nx.Graph):
    ax.clear()

    node_colors = [G.nodes[node]['color'] for node in G.nodes()]
    edge_colors = [G.edges[edge]['color'] for edge in G.edges()]
    nx.draw(
        G,
        pos=pos,
        with_labels=True,
        node_color=node_colors,
        edge_color=edge_colors,
        node_size=800,
        font_weight='bold',
        ax=ax
    )

    for node in G.nodes():
        if 'resources' in G.nodes[node]:
            resources = G.nodes[node]['resources']
            ax.text(
                pos[node][0],
                pos[node][1] + 0.15,
                resources,
                horizontalalignment='center',
                verticalalignment='center'
            )

def clear(G: nx.Graph):
    for node in G.nodes():
        if 'color' in G.nodes[node]:
            G.nodes[node]['color'] = 'lightblue'
    for edge in G.edges():
        if 'color' in G.edges[edge]:
            G.edges[edge]['color'] = 'black'
    draw(G)

draw(G)
plt.show()
plt.ion()


In [4]:
start_node: str = 'n1'  # Replace with your preferred start node
resource: str = 'r15'  # Replace with your preferred end node

algorithm_dropdown = widgets.Dropdown(
    options=['Flooding', 'Random Walk'],
    value='Flooding',
    description='Algorithm:'
)

start_node_text = widgets.Text(value=start_node, description='Start Node:')
resource_text = widgets.Text(value=resource, description='Resource:')

cache_checkbox = widgets.Checkbox(value=True, description='Enable Cache')

max_ttl_text = widgets.Text(value=str(sys.maxsize), description='Max TTL:')

run_button = widgets.Button(description='Run')
clear_cache_button = widgets.Button(description='Clear Cache')
show_cache_button = widgets.Button(description='Show Cache')

output = widgets.Output()

def run_algorithm(_):
    clear(G)
    clear_output()
    display(widgets_box, output)
    algorithm = algorithm_dropdown.value
    start = start_node_text.value
    res = resource_text.value
    use_cache = cache_checkbox.value
    max_ttl = int(max_ttl_text.value)

    with output:
        output.clear_output()
        if algorithm == 'Flooding':
            print("Running Flooding from node", start, "to resource", res)
            connection_count = flooding(
                G,
                start,
                res,
                draw,
                cache_bfs if use_cache else None,
                max_ttl=max_ttl
            )
            print("connection_count:", connection_count)
        elif algorithm == 'Random Walk':
            print("Running Random Walk from node", start, "to resource", res)
            connection_count = random_walk(
                G,
                start,
                res,
                draw,
                cache_dfs if use_cache else None,
                max_ttl=max_ttl
            )
            print("connection_count:", connection_count)
            
def clear_cache():
    global cache_dfs
    global cache_bfs
    cache_dfs = {}
    cache_bfs = {}
    clear_output()
    display(widgets_box, output)


def show_cache():
    print('Cache DFS: ', cache_dfs)
    print('Cache BFS: ', cache_bfs)

run_button.on_click(run_algorithm)
clear_cache_button.on_click(lambda _: clear_cache())
show_cache_button.on_click(lambda _: show_cache())

widgets_box = widgets.VBox(
    [algorithm_dropdown, start_node_text, resource_text, cache_checkbox, max_ttl_text, run_button, clear_cache_button, show_cache_button]
)

display(widgets_box, output)


Output(outputs=({'output_type': 'stream', 'text': "Running Flooding from node n1 to resource r6\nCACHE HIT! Br…

Cache DFS:  {}
Cache BFS:  {('n2', 'r6'): ['n2', 'n4'], ('n1', 'r6'): ['n1', 'n2', 'n4'], ('n3', 'r6'): ['n3', 'n1', 'n2', 'n4']}


In [5]:
import time
from tqdm import tqdm

num_runs_text = widgets.Text(value='1000', description='Runs:')
run_all_button = widgets.Button(description='Run All')
output_results = widgets.Output()
progress_bar = widgets.IntProgress(value=0, min=0, max=1, description='Progress:', bar_style='info')

def run_all_runs(_):
    num_runs = int(num_runs_text.value)
    values = []
    
    with output_results:
        output_results.clear_output()
        progress_bar.max = num_runs
        display(progress_bar)
        
        for _ in tqdm(range(num_runs), desc='Running'):
            path, connection_count = instant_random_walk(G, start_node, resource)
            values.append(connection_count)
            clear(G)
            time.sleep(0.01)  # Adding a small delay for progress bar visualization
            
            progress_bar.value += 1
        
        # Calculate statistics
        min_val = min(values)
        max_val = max(values)
        mean_val = sum(values) / len(values)
        median_val = sorted(values)[len(values) // 2]
        std_val = (sum((x - mean_val) ** 2 for x in values) / len(values)) ** 0.5
        
        print("min:", min_val)
        print("max:", max_val)
        print("mean:", mean_val)
        print("median:", median_val)
        print("std:", std_val)
        
        progress_bar.value = 0  # Reset the progress bar after completion

run_all_button.on_click(run_all_runs)

run_block = widgets.VBox([num_runs_text, run_all_button])
results_block = widgets.VBox([output_results])
progress_block = widgets.VBox([progress_bar])

display(run_block, progress_block, results_block)
